Detects if a Tezos block is a Flashbake block by looking whether the operation order matches the standard baker's heuristic.

In [1]:
import requests
import json
import sys

def is_fb_block(level):
    ops = requests.get(f"https://mainnet.oxheadhosted.com/chains/main/blocks/{level}/operations/3?metadata=never").json()
    simplified_ops = []
    for op_num,op in enumerate(ops):
        sop = { "contents": []}
        for c in op["contents"]:
            sc = { "fee": c["fee"],
                    "gas_limit": c["gas_limit"]}
            sop["contents"].append(sc)
        sop["fee_sum"] = sum([int(c["fee"]) for c in op["contents"]])
        sop["operation_gas"] = sum([int(c["gas_limit"]) for c in op["contents"]])
        sop["gas_ratio"] = sop["operation_gas"] / 5200000
        # operation size in bytes. We perform a binary http query and substract 68 bytes metadata
        # NOPE ! 32 (protocol hash) + 32 (hash de l'op)  + 4 (chain id) => faut enlever 68 bytes
        # 32 (hash de l'op) + 4 (chain id) + 13 (??) => faut enlever 49 bytes
        extralen=49
        sop["operation_size"] = len(bytes(requests.get(f"https://mainnet.oxheadhosted.com/chains/main/blocks/{level}/operations/3/{op_num}?metadata=never", headers={'accept': 'application/octet-stream'}).content)) - extralen
        sop["size_ratio"] = sop["operation_size"] / (512*1024)
        sop["weight"] = sop["fee_sum"] / max(sop["size_ratio"], sop["gas_ratio"])
        sop["hash"] = op["hash"][0:6]
        simplified_ops.append(sop)
#print(json.dumps(simplified_ops, indent=2))
    weights = [ int(op["weight"]) for op in simplified_ops]
    if (sorted(weights, reverse = True) == weights):
        print(f"Block {level} is not a FB block")
    else:
        print("operation_hash,operation_size,operation_gas,fee_sum,weight")
        for op in simplified_ops:
            print(f"{op['hash']},{op['operation_size']},{op['operation_gas']},{op['fee_sum']},{op['weight']}")
        print(f"Block {level} is a Flashbake block!")
        exit(1)


is_fb_block(2794842)

Block 2794842 is not a FB block
